# The Product Pricer Continued...

## Testing Gemini-1.5-pro model

In [0]:
import os
import re
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pickle
import google.generativeai as google_genai
import time

In [0]:
from items import Item
from testing import Tester 

In [0]:
# environment
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [0]:
google_genai.configure()

In [0]:
%matplotlib inline

In [0]:
# Load in the pickle files that are located in the `pickled_dataset` folder
with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [0]:
# Function to create the messages format required for Gemini 1.5 Pro
# This function prepares the system and user messages in the format expected by Gemini models.
def gemini_messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    
    # Modify the test prompt by removing "to the nearest dollar" and "Price is $"
    # This ensures that the model receives a cleaner, simpler prompt.
    user_prompt = item.test_prompt().replace(" to the nearest dollar", "").replace("\n\nPrice is $", "")

    # Reformat messages to Gemini’s expected format: messages = [{'role':'user', 'parts': ['hello']}]
    return [
        {"role": "system", "parts": [system_message]},  # System-level instruction
        {"role": "user", "parts": [user_prompt]},       # User's query
        {"role": "model", "parts": ["Price is $"]}  # Assistant's expected prefix for response
    ]

In [0]:
# Example usage of the gemini_messages_for function
gemini_messages_for(test[0])  # Generate message structure for the first test item

In [0]:
# Utility function to extract the numerical price from a given string
# This function removes currency symbols and commas, then extracts the first number found.
def get_price(s):
    s = s.replace('$', '').replace(',', '')  # Remove currency symbols and formatting
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)  # Regular expression to find a number
    return float(match.group()) if match else 0  # Convert matched value to float, return 0 if no match

In [0]:
# Example usage of get_price function
get_price("The price is roughly $99.99 because blah blah")  # Expected output: 99.99

In [0]:
# Function to get the estimated price using Gemini 1.5 Pro
def gemini_1_point_5_pro(item):
    messages = gemini_messages_for(item)  # Generate messages for the model
    system_message = messages[0]['parts'][0]  # Extract system-level instruction
    user_messages = messages[1:]  # Remove system message from messages list
    
    # Initialize Gemini 1.5 Pro model with system instruction
    gemini = google_genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        system_instruction=system_message
    )

    # Generate response using Gemini API
    response = gemini.generate_content(
        contents=user_messages,
        generation_config=google_genai.GenerationConfig(max_output_tokens=5)
    )

    # Extract text response and convert to numerical price
    return get_price(response.text)

In [0]:
# Example usage:
gemini_1_point_5_pro(test[0])

In [0]:
# Retrieve the actual price of the test item (for comparison)
test[0].price    # Output: 374.41

In [0]:
# Test the function for gemini-1.5 pro using the Tester framework
Tester.test(gemini_1_point_5_pro, test)

## Five, Gemini-2.0-flash

In [0]:
# Function to get the estimated price using Gemini-2.0-flash-exp
def gemini_2_point_0_flash_exp(item):
    messages = gemini_messages_for(item)  # Generate messages for the model
    system_message = messages[0]['parts'][0]  # Extract system-level instruction
    user_messages = messages[1:]  # Remove system message from messages list
    
    # Initialize Gemini-2.0-flash-exp model with system instruction
    gemini = google_genai.GenerativeModel(
        model_name="gemini-2.0-flash-exp",
        system_instruction=system_message
    )

    # Adding a delay to avoid hitting the API rate limit and getting a "ResourceExhausted: 429" error
    time.sleep(5)
    
    # Generate response using Gemini API
    response = gemini.generate_content(
        contents=user_messages,
        generation_config=google_genai.GenerationConfig(max_output_tokens=5)
    )

    # Extract text response and convert to numerical price
    return get_price(response.text)

In [0]:
# Example usage:
gemini_2_point_0_flash_exp(test[0])  

In [0]:
# Retrieve the actual price of the test item (for comparison)
test[0].price    # Output: 374.41

In [0]:
# Test the function for gemini-2.0-flash-exp using the Tester framework
Tester.test(gemini_2_point_0_flash_exp, test)